In [1]:
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize

from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.applications import ResNet50
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.resnet50 import preprocess_input

In [2]:
directory = r'C:\Users\Deepak\Downloads\id_card detector\ID_Card data set'
categories = ['without ID cards', 'with ID cards']

data=[]
labels=[]

In [3]:
for category in categories:
  path = os.path.join(directory, category)
  for img in os.listdir(path):
    img_path = os.path.join(path, img)
    image = load_img(img_path,target_size=(224,224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)
    labels.append(category)

In [4]:
labels = label_binarize(labels,classes=categories)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(np.array(data), labels, test_size=0.2, random_state=42)

In [6]:
idg=keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True
)

In [7]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

m = base_model.output
m = GlobalAveragePooling2D()(m)
m = Dense(512, activation='relu')(m)
m = Dropout(0.5)(m)
predictions = Dense(1, activation='sigmoid')(m)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

In [8]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
bs=32
model.fit(
        idg.flow(X_train,y_train,bs),
        steps_per_epoch=len(X_train)//bs,
        epochs=20,
        validation_data=(X_test,y_test),
        validation_steps=len(X_test)//bs)

Epoch 1/20
6/6 [==============================] - 34s 4s/step - loss: 1.5732 - accuracy: 0.7030 - val_loss: 0.3516 - val_accuracy: 0.8800
Epoch 2/20
6/6 [==============================] - 23s 4s/step - loss: 0.3312 - accuracy: 0.8606 - val_loss: 0.4947 - val_accuracy: 0.8400
Epoch 3/20
6/6 [==============================] - 24s 4s/step - loss: 0.2054 - accuracy: 0.9333 - val_loss: 0.4146 - val_accuracy: 0.8800
Epoch 4/20
6/6 [==============================] - 23s 4s/step - loss: 0.1155 - accuracy: 0.9576 - val_loss: 0.2237 - val_accuracy: 0.9600
Epoch 5/20
6/6 [==============================] - 22s 4s/step - loss: 0.1257 - accuracy: 0.9636 - val_loss: 0.4126 - val_accuracy: 0.8800
Epoch 6/20
6/6 [==============================] - 21s 4s/step - loss: 0.1054 - accuracy: 0.9758 - val_loss: 0.2105 - val_accuracy: 0.9600
Epoch 7/20
6/6 [==============================] - 21s 4s/step - loss: 0.1791 - accuracy: 0.9515 - val_loss: 0.2149 - val_accuracy: 0.9600
Epoch 8/20
6/6 [==================

In [10]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy on test set: {accuracy}')

2/2 [==============================] - 3s 690ms/step - loss: 0.0824 - accuracy: 0.9600
Accuracy on test set: 0.9599999785423279


In [11]:
model.save("id_card_detect.model", save_format="h5")

c:\Users\Deepak\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
